In [1]:
import importlib
import json
import pickle
import matplotlib.pyplot as plt
import scipy.stats as stats

from generate_dataset import *
from neural_net import *
from plotting import *
from postprocessing_dataset import *
from predicting import *
from preprocessing_dataset import *
from testing import *

experiment_1 = importlib.import_module("experiment 1")
importlib.reload(experiment_1)

generate_dataset = importlib.import_module("generate_dataset")
importlib.reload(generate_dataset)

preprocessing_datasetv = importlib.import_module("preprocessing_dataset")
importlib.reload(preprocessing_datasetv)


postprocessing_datasetv = importlib.import_module("postprocessing_dataset")
importlib.reload(postprocessing_datasetv)

imported_module = importlib.import_module("testing")
importlib.reload(imported_module)




<module 'testing' from 'f:\\Jelena\\FAKULTET\\Student\\DOKTORAT\\projekat 2rad\\oop\\testing.py'>

In [ ]:
def f1_score(prec,recall):
    prec = np.asarray(prec).mean(axis = 1)
    rec = np.asarray(recall).mean(axis = 1)
    f1 = np.nan_to_num(2*prec*rec/(prec+rec))
    return f1

In [ ]:

def overlaping_predictions(predictions, features):
    overll = []
    distance = []
    for idx in range(0,predictions.shape[0]):
        obj = features[idx, 1, :]
        # overlaping = np.logical_and(predictions[idx], obj).sum()

        length02  = np.where(predictions[idx] == 1)[0]
        lengtho1 = np.where(features[idx, 1, :] == 1)[0]
        # print(length02[0] -  lengtho1[-1])
        # print(lengtho1[0] -  length02[-1])
        dist = 0
        overlaping = 0

        if length02.sum() != 0:
            dist =  max([length02[0] -  lengtho1[-1], lengtho1[0] -  length02[-1]])


            if dist <0 :
                overlaping = -dist
                dist = -1


            elif dist > 0:
                overlaping = -1


        distance.append(dist)
        overll.append(overlaping)
    return overll,distance




def overlapping_type(Y_pred, X,obj_id,length_feature = 1):### 
    """
    za odredjeni objekat za svaki trening primer proverava koliko se sece sa objektima ciji je id manji od njegovog
    i za svaki tr_primer vraca kolike je overlaping sa svakim od prethodnih 
    ----ostalima, dakle vraca samo jedan broj
    """
    overll_per_example = []
    distanc_per_example = []### za svaki trening primer gledam da li se preseko sa tipovima pre njega
      

    for i in range(0,X.shape[0]):#za svaki trening primer

        pred= Y_pred[i]
        j = obj_id-1# overlaping sa tipom pre njega
        obj = X[i,length_feature+j,:]
 
        
        length02  = np.where(pred == 1)[0]
        lengtho1 = np.where(obj == 1)[0]

        if lengtho1.sum() == 0:
            lengtho1 = [0]
        dist = 0
        overlaping = 0

        if length02.sum() != 0:
                dist =  max([length02[0] -  lengtho1[-1], lengtho1[0] -  length02[-1]])


                if dist <0 :
                    overlaping = -dist
                    dist = -1


                elif dist > 0:
                    overlaping = -1


        distanc_per_example.append(dist)
        overll_per_example.append(overlaping)
    return np.asarray(overll_per_example),np.asarray(distanc_per_example)



def overlaping_predictions(predictions, features):
    ### ne gleda overlaping z aprvi objekat posto je on svakako nula jer pre njega nema nikoga
        testset_size = 1000
        object_types_prediction  = list(range(0,5))
        pred_types, X_types = split_dataset_by_type(np.asarray(predictions), testset_size),split_dataset_by_type(np.asarray(features), testset_size)
        # pred_types, X_types = split_dataset_by_type(np.asarray(pred_pip[0]), testset_size)[1:],split_dataset_by_type(np.asarray(X_predictions[0]), testset_size)[1:]
        print(X_types.shape)
        print(pred_types.shape)

        overlaping = []   
        distance = []
        for idx in object_types_prediction[1:]:
            overlap,dist = overlapping_type( np.asarray(pred_types)[idx],np.asarray(X_types)[idx],idx,0)
            overlaping.append(overlap)
            distance.append(dist)
        return  np.asarray(distance), np.asarray(overlaping)
    
 

In [ ]:
import numpy as np
def split_dataset_by_type(dataset, interval):
    splitted = []
    for i in range(0, int(dataset.shape[0]/interval)):
        start = i*interval
        end = start + interval
        end = end if end < dataset.shape[0] else  dataset.shape[0]
        splitted.append(dataset[start:end])
    return np.asarray(splitted)

In [3]:
    
    import importlib
    experiment_1 = importlib.import_module("experiment 1")
    importlib.reload(experiment_1)
    import importlib
    generate_dataset = importlib.import_module("generate_dataset")
    importlib.reload(generate_dataset)

    predicting = importlib.import_module("predicting")
    importlib.reload(predicting)
    
    
    postprocessing_datasetv = importlib.import_module("postprocessing_dataset")
    importlib.reload(postprocessing_datasetv)
    #def experiment_per_iteration(data_train,data_test, train_set_sizes,testset_size, max_object_types) = (data_train,data_test,[100,200,500,1000,4000],1000,10)

    def experiment_per_iteration(data_train,data_test, train_set_sizes,testset_size, max_object_types):
    
        def reset_weights(model):

            import keras.backend as K
            session = K.get_session()
            for layer in model.layers: 
                if hasattr(layer, 'kernel_initializer'): 
                    layer.kernel.initializer.run(session=session)
                if hasattr(layer, 'bias_initializer'):
                    layer.bias.initializer.run(session=session)
        max_space_length = 600

        length_feature = 0
        precissions_types_pip, recall_types_pip,precissions_types, recall_types = [],[],[],[]
        pipeline_predictions, separate_predictions = [],[]

        X_predictions = []

        f1_score_pip, f1_score_seq = [],[]
        # object_types_prediction  = list(range(0,10))
        train_raw_dataset = experiment_1.make_train(data_train, train_set_sizes[0])
        X_test,Y_test = experiment_1.data_preprocessing(data_test,max_object_types)
        dist_overl_pip, dist_overlap_seq = [],[]

        for train_size_id in range(len(train_set_sizes)):
            if train_size_id > 0:        
                train_raw_dataset = experiment_1.make_train(data_train, train_set_sizes[train_size_id])
                reset_weights(model)

            X_train,Y_train = experiment_1.data_preprocessing(train_raw_dataset,max_object_types)
            X_test_types, Y_types =split_dataset_by_type(X_test, testset_size),split_dataset_by_type(Y_test, testset_size)
            object_types_prediction  = list(range(0,max_object_types))
            print("y_shape",Y_types.shape)

            model = run_nn((X_train, Y_train), (X_test, Y_test),5 )
            predictions_pip_types,predictions_features = predicting.predictions_pipeline(X_test_types,model,object_types_prediction,Y_types, 0)


            print(np.asarray(predictions_pip_types).shape)
            predictions_pip = np.concatenate(predictions_pip_types,0)

            predictions_features = np.concatenate(predictions_features)

    #         precision_train_types, recall_train_types = precisions_recalls_types_ex(pred_train,Y_train,object_types_prediction)

            precision_test_types, recall_test_types = precisions_recalls_types_ex(predictions_pip,Y_test,object_types_prediction)
    #         precissions_types_train.append(precision_train_types)
    #         recall_types_train.append(recall_train_types)

            f1_score_pip.append(f1_score(precision_test_types,recall_test_types))
            dist_overl_pip.append([overlaping_predictions(predictions_pip,predictions_features)])


            ###sequential


            test_conv_nn = X_test.reshape(-1, 1, max_space_length, max_object_types*2+length_feature)
            pred = model.predict(test_conv_nn,max_object_types)
            # pred1 = pred>0.5

            print(pred.shape)
            print(Y_test.shape)
            pred1 = experiment_1.arrange_object(pred, Y_test)

            precisions, recalls = precisions_recalls_types_ex(pred1,Y_test,object_types_prediction)

            f1_score_seq.append(f1_score(precisions,recalls))
            dist_overlap_seq.append([overlaping_predictions(pred1,X_test)])

        return f1_score_pip, dist_overl_pip,f1_score_seq,dist_overlap_seq

In [ ]:
# ### verzija van funkcije
# (data_train,data_test, train_set_sizes,testset_size, max_object_types) = (data_train,data_test,[1000],1000,10)


# def reset_weights(model):

#     import keras.backend as K
#     session = K.get_session()
#     for layer in model.layers: 
#         if hasattr(layer, 'kernel_initializer'): 
#             layer.kernel.initializer.run(session=session)
#         if hasattr(layer, 'bias_initializer'):
#             layer.bias.initializer.run(session=session)
# max_space_length = 600

# length_feature = 0
# precissions_types_pip, recall_types_pip,precissions_types, recall_types = [],[],[],[]
# pipeline_predictions, separate_predictions = [],[]

# X_predictions = []

# f1_score_pip, f1_score_seq = [],[]
# # object_types_prediction  = list(range(0,10))
# train_raw_dataset = experiment_1.make_train(data_train, train_set_sizes[0])
# X_test,Y_test = experiment_1.data_preprocessing(data_test,max_object_types)
# dist_overl_pip, dist_overlap_seq = [],[]

# for train_size_id in range(len(train_set_sizes)):
#     if train_size_id > 0:        
#         train_raw_dataset = experiment_1.make_train(data_train, train_set_sizes[train_size_id])
#         reset_weights(model)

#     X_train,Y_train = experiment_1.data_preprocessing(train_raw_dataset,max_object_types)
#     X_test_types, Y_types =split_dataset_by_type(X_test, testset_size),split_dataset_by_type(Y_test, testset_size)
#     object_types_prediction  = list(range(0,max_object_types))
#     print("y_shape",Y_types.shape)

#     model = run_nn((X_train, Y_train), (X_test, Y_test),5 )
#     predictions_pip_types,predictions_features = predicting.predictions_pipeline(X_test_types,model,object_types_prediction,Y_types, 0)


#     print(np.asarray(predictions_pip_types).shape)
#     predictions_pip = np.concatenate(predictions_pip_types,0)

#     predictions_features = np.concatenate(predictions_features)

# #         precision_train_types, recall_train_types = precisions_recalls_types_ex(pred_train,Y_train,object_types_prediction)

#     precision_test_types, recall_test_types = precisions_recalls_types_ex(predictions_pip,Y_test,object_types_prediction)
# #         precissions_types_train.append(precision_train_types)
# #         recall_types_train.append(recall_train_types)

#     f1_score_pip.append(f1_score(precision_test_types,recall_test_types))
#     dist_overl_pip.append([overlaping_predictions(predictions_pip,predictions_features)])


#     ###sequential


#     test_conv_nn = X_test.reshape(-1, 1, max_space_length, max_object_types*2+length_feature)
#     pred = model.predict(test_conv_nn,max_object_types)
#     # pred1 = pred>0.5

#     print(pred.shape)
#     print(Y_test.shape)
#     pred1 = experiment_1.arrange_object(pred, Y_test)

#     precisions, recalls = precisions_recalls_types_ex(pred1,Y_test,object_types_prediction)

#     f1_score_seq.append(f1_score(precisions,recalls))
#     dist_overlap_seq.append([overlaping_predictions(pred1,X_test)])

       

In [4]:
results = []
for i in range(0,1):
    data = experiment_1.import_dataset("data_B.json")
    data_train,data_test = experiment_1.make_train_test(data, 4000, 1000)
    results.append(experiment_per_iteration(data_train,data_test,[100, 200,500,1000,4000],1000,5))

# for i in range(0,5):
#     print(i)
#     with open("exp_2_results_2pipseq"+str(i), "wb") as fp:   #Pickling
#         pickle.dump(results[i], fp)

broj features
10
max
5
broj features
10
max
5
y_shape (5, 1000, 600)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1, 596, 20)        1020      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 1, 596, 20)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 592, 20)        2020      
                                                                 
 re_lu (ReLU)                (None, 1, 592, 20)        0         
                                                                 
 flatten (Flatten)           (None, 11840)             0         
                                                                 
 dense (Dense)               (None, 270)               3197070   
                                                     

f:\Jelena\FAKULTET\Student\DOKTORAT\venv\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/5
16/16 [==============================] - 6s 244ms/step - loss: 0.2804 - binary_accuracy: 0.8845 - accuracy: 0.0640 - categorical_accuracy: 0.0640 - val_loss: 0.1408 - val_binary_accuracy: 0.9359 - val_accuracy: 0.0070 - val_categorical_accuracy: 0.0070
Epoch 2/5
16/16 [==============================] - 4s 269ms/step - loss: 0.0999 - binary_accuracy: 0.9547 - accuracy: 0.0060 - categorical_accuracy: 0.0060 - val_loss: 0.0753 - val_binary_accuracy: 0.9661 - val_accuracy: 0.0036 - val_categorical_accuracy: 0.0036
Epoch 3/5
16/16 [==============================] - 2s 137ms/step - loss: 0.0594 - binary_accuracy: 0.9738 - accuracy: 0.1200 - categorical_accuracy: 0.1200 - val_loss: 0.0605 - val_binary_accuracy: 0.9740 - val_accuracy: 0.1518 - val_categorical_accuracy: 0.1518
Epoch 4/5
16/16 [==============================] - 2s 123ms/step - loss: 0.0457 - binary_accuracy: 0.9801 - accuracy: 0.1460 - categorical_accuracy: 0.1460 - val_loss: 0.0519 - val_binary_accuracy: 0.9773 - val_

: 

In [ ]:
# results = []
# # dump information to that file
# for i in range(0,5):
#     print(i)
#     file = open("F:/Jelena/FAKULTET/Student/DOKTORAT/projekat 2rad/oop/exp_2_results_2pipseq"+str(i), 'rb')
#     results.append(pickle.load(file))

In [1]:
def mean_iteration(data,measure_id):
    mean_size_id = []
    for train_size_id in range(5):
        mean = []
        for iteraions in range(5):
            mean.append(data[iteraions][measure_id][train_size_id])
        mean_size_id.append(np.asarray(mean).mean(axis = 0))
        
    return mean_size_id

f1_measure = mean_iteration(results,0)

for i in range(5):
    plt.plot(f1_measure[i])
plt.legend(["100","200","500","1000","4000"])
np.asarray(f1_measure).mean(axis = 1)*100

NameError: name 'results' is not defined

In [ ]:
np.savetxt('dataaa.dat', np.around(np.asarray(f1_measure)*100, decimals=2),fmt='%.2f',delimiter=',')

### Overlaping distance


In [ ]:

distover = []
for i in range(5):
    distover.append(results[i][1])
distover = np.asarray(distover).squeeze()
dist = distover[:,:,0,:,:]
over = distover[:,:,1,:,:]

In [ ]:
### ssrednji overlapihng za prmere koji overlapuju
a = np.ma.masked_where(over<=0,over).mean(axis = 0).mean(axis =2).data
np.savetxt('dataaa.dat', np.around(np.asarray(a), decimals=2),fmt='%.2f',delimiter=',')

In [ ]:
### ssrednji dist za prmere koji ne overlauju
dista = np.ma.masked_where(dist<=0,dist).mean(axis = 0).mean(axis =2).data
np.savetxt('dataaa.dat', np.around(np.asarray(dista), decimals=2),fmt='%.2f',delimiter=',')

In [ ]:
### za exp B koliko ih bas na tacnom razmkau stavi
#((dist%10 == 0) & (dist >0)).mean(axis = 0).mean(axis =2)*100

In [ ]:
### koliki procenat ne overlepuje
a = (over==0).mean(axis = 0).mean(axis = 2)
np.savetxt('dataaa.dat', np.around(np.asarray(a)*100, decimals=2),fmt='%.2f',delimiter=',')